In [ ]:
import cv2
import mediapipe as mp
import time
import tkinter as tk
from tkinter import filedialog, messagebox

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def process_video(input_path, output_path, scale_factor=1.0, skip_frames=1,
                  rotate=False, draw_landmarks=True, model_complexity=1):
    cap = cv2.VideoCapture(input_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # חישוב פרופורציות לפי רוטציה
    if rotate:
        new_width = int(height * scale_factor)
        new_height = int(width * scale_factor)
    else:
        new_width = int(width * scale_factor)
        new_height = int(height * scale_factor)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (new_width, new_height))

    print(f"עיבוד וידאו ({total_frames} פריימים)...")

    with mp_pose.Pose(
        static_image_mode=False,
        model_complexity=model_complexity,
        enable_segmentation=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as pose:

        frame_num = 0
        start_time = time.time()

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            frame_num += 1
            if frame_num % skip_frames != 0:
                continue

            if rotate:
                frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

            frame = cv2.resize(frame, (new_width, new_height))
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            if results.pose_landmarks and draw_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0,0,255), thickness=2))

            progress_text = f"Frame {frame_num}/{total_frames}"
            cv2.putText(frame, progress_text, (30, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

            out.write(frame)
            cv2.imshow('Pose Output (Q to quit)', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        out.release()
        cv2.destroyAllWindows()

        duration = time.time() - start_time
        print(f"נשמר ב: {output_path}, זמן ריצה: {duration:.1f} שניות")

def run_gui():
    def select_input():
        path = filedialog.askopenfilename()
        entry_input.delete(0, tk.END)
        entry_input.insert(0, path)

    def select_output():
        path = filedialog.asksaveasfilename(defaultextension=".mp4")
        entry_output.delete(0, tk.END)
        entry_output.insert(0, path)

    def start_processing():
        try:
            input_path = entry_input.get()
            output_path = entry_output.get()
            scale = float(scale_var.get())
            skip = int(skip_var.get())
            rotate = rotate_var.get()
            draw = draw_var.get()
            complexity = int(complexity_var.get())

            process_video(
                input_path, output_path,
                scale_factor=scale,
                skip_frames=skip,
                rotate=rotate,
                draw_landmarks=draw,
                model_complexity=complexity
            )
        except Exception as e:
            messagebox.showerror("Error", str(e))

    root = tk.Tk()
    root.title("Pose Video Optimizer")

    tk.Label(root, text="Input Video:").grid(row=0, column=0, sticky='e')
    entry_input = tk.Entry(root, width=40)
    entry_input.grid(row=0, column=1)
    tk.Button(root, text="Browse", command=select_input).grid(row=0, column=2)

    tk.Label(root, text="Output Video:").grid(row=1, column=0, sticky='e')
    entry_output = tk.Entry(root, width=40)
    entry_output.grid(row=1, column=1)
    tk.Button(root, text="Browse", command=select_output).grid(row=1, column=2)

    tk.Label(root, text="Resize Factor (0.1–1.0):").grid(row=2, column=0, sticky='e')
    scale_var = tk.StringVar(value="1.0")
    tk.Entry(root, textvariable=scale_var).grid(row=2, column=1)

    tk.Label(root, text="Frame Skip (1–10):").grid(row=3, column=0, sticky='e')
    skip_var = tk.StringVar(value="1")
    tk.Entry(root, textvariable=skip_var).grid(row=3, column=1)

    rotate_var = tk.BooleanVar()
    tk.Checkbutton(root, text="Rotate 90°", variable=rotate_var).grid(row=4, column=1, sticky='w')

    draw_var = tk.BooleanVar(value=True)
    tk.Checkbutton(root, text="Draw Landmarks", variable=draw_var).grid(row=5, column=1, sticky='w')

    tk.Label(root, text="Model Complexity (0–2):").grid(row=6, column=0, sticky='e')
    complexity_var = tk.StringVar(value="1")
    tk.Entry(root, textvariable=complexity_var).grid(row=6, column=1)

    tk.Button(root, text="Start", command=start_processing, bg='green', fg='white').grid(row=7, column=1, pady=10)

    root.mainloop()

if __name__ == "__main__":
    run_gui()


עיבוד וידאו (1806 פריימים)...


I0000 00:00:1754307504.869808 20958187 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1
W0000 00:00:1754307504.962855 20966877 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754307504.976133 20966880 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


נשמר ב: /Users/shay_ziv/Downloads/elapose.mp4, זמן ריצה: 19.7 שניות
